## 事前準備

In [ ]:
 # google driveへの接続

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CONST
定数の宣言  
ここは各自の環境に合わせて変更してください。ここではGoogleDriveを用いて行うことにします。  
ルートフォルダ内にdataフォルダを作成し、そこにダウンロードしたzipファイルを入れていください。

In [6]:
## rootフォルダ
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb"

## 出力フォルダ
OUTPUT_DIR = ROOT_DIR+ "/output"

## DATA
DATE_DIR = "/data"

etl9b_path = '/content/drive/MyDrive/code/ETLCDB/ETL9B.zip' # ここにetl9bのパスを入力

In [ ]:
### フォルダ作成
import os
#### 上記のフォルダがなかった場合、作成する処理をここに入れる

## 処理

### import

In [10]:
### import
import shutil

import struct
from PIL import Image, ImageEnhance

import glob
import os

### make pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import pickle

### unzip

In [ ]:


shutil.unpack_archive(etl9b_path ,OUTPUT_DIR+"/unzip")

### binary -> png

In [ ]:

dataset_dir = OUTPUT_DIR + '/unzip'

files = glob.glob(dataset_dir + '/ETL9B/*') # 解凍したバイナリファイルのpath

err_cnt = 0
for fname in files:
    print('===== ===== =====')
    if fname == dataset_dir + "/ETL9B/ETL9BINFO": continue # 情報ファイルは飛ばす
    if fname == dataset_dir + "/ETL9B/output": continue

    ### まとめて解凍すると、時間かかるから分割する。
    # if fname == dataset_dir + "/ETL9B/ETL9B_1": continue 
    # if fname == dataset_dir + "/ETL9B/ETL9B_2": continue 
    if fname == dataset_dir + "/ETL9B/ETL9B_3": continue 
    if fname == dataset_dir + "/ETL9B/ETL9B_4": continue 
    if fname == dataset_dir + "/ETL9B/ETL9B_5": continue 

    print(fname)

    f = open(fname, 'rb')
    # f.seek(0) # ポインタを先頭に
    f.seek(576)
    cnt = 0
    while True:
        cnt += 1
        print('----- -----　-----')
        print(fname +":::" +str(cnt))
        s = f.read(576) 
        if not s: break
        # バイナリデータなのでPythonが理解できるように抽出
        try:
          r = struct.unpack('>2H4s504s64x', s)
        except:
          err_cnt += 1
          continue
        kanjicode_jis = r[1]

        # 画像を抽出
        iF = Image.frombytes('1', (64, 63), r[3], 'raw')
        iP = iF.convert('L')
        dir = OUTPUT_DIR+ "/etl9b/{:x}".format(kanjicode_jis)
        print(dir)
        if not os.path.exists(dir): 
          os.makedirs(dir)

        #####
        print("r[0]: " + str(r[0]))
        print("r[1]: " + str(r[1]))
        print("r[2]: " + str(r[2]))
        #####
        fn = "{0:02x}-{1:02x}{2}.png".format(kanjicode_jis, r[0], r[2])
        print('fn: ' + fn)
        #####

        fullpath = dir + "/" + fn
        if os.path.exists(fullpath): continue
        enhancer = ImageEnhance.Brightness(iP)
        iE = enhancer.enhance(16)
        iE.save(fullpath, 'PNG')

print("CNT: "+ str(cnt))
print("ERR: "+ str(err_cnt))

### make-pickle

In [ ]:
# インライン表示
%matplotlib inline

# 保存ディレクトリと画像サイズの指定
out_dir = OUTPUT_DIR + '/etl9b'
im_size = 64 # 画像サイズ

# 保存ファイル名と保存先
save_file = OUTPUT_DIR + "/pickle/ETL9B.dill" 
# save_file = OUTPUT_DIR + "/pickle/ETL9B.pickle" 

# 画像ディレクトリから画像を読み込み開始
files = os.listdir(out_dir)
files_dir = [f for f in files if os.path.isdir(os.path.join(out_dir, f))]

result = []

print(files_dir)
for i, code in enumerate(files_dir):
    img_dir = out_dir + "/" + str(code)
    fs = glob.glob(img_dir + "/*")
    print("dir=",  img_dir)

    # test
    print("CODE: " + code)
    print(len(result))

    # 画像64X63を読み込んでグレイスケールに変換し64X64に整形
    for j, f in enumerate(fs):
      try:
        img = cv2.imread(f)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img_gray, (im_size, im_size))
        result.append([i, img])

        ### show image
        # print(code + ": "+ str(j))
        # plt.imshow(img)
        # plt.show()

      except :
        print("ERROR: "+ str(j))


# 「画像とラベル」のデータセットを保存
pickle.dump(result, open(save_file, "wb"))

In [5]:
# pickleが使用できないとき
!pip install dill
import dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 保存

dill_file = OUTPUT_DIR + "/pickle/ETL9B.dill" 
pickle_file = OUTPUT_DIR + "/pickle/ETL9B.pickle" 

try:
  with open(pickle_file, "wb") as save:
    pickle.dump(result, save)
except:
  with open(dill_file, "wb") as save:
    dill.dump(result, save)

#### make pickle(sub)
同時にdampするversion

In [ ]:
# 保存ディレクトリと画像サイズの指定
out_dir = OUTPUT_DIR + '/etl9b'
im_size = 64 # 画像サイズ

save_file = OUTPUT_DIR + "/pickle/ETL9B.pickle" # 保存ファイル名と保存先
plt.figure(figsize=(9, 17)) 

# 画像ディレクトリから画像を読み込み開始
files = os.listdir(out_dir)
files_dir = [f for f in files if os.path.isdir(os.path.join(out_dir, f))]

result = []
for i, code in enumerate(files_dir):
    img_dir = out_dir + "/" + str(code)
    fs = glob.glob(img_dir + "/*")
    print("dir=",  img_dir)

    # 画像64X63を読み込んでグレイスケールに変換し64X64に整形
    for j, f in enumerate(fs):
      try:
        img = cv2.imread(f)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img_gray, (im_size, im_size))
        result.append([i, img])
        try:
          pickle.dump(result, open(save_file, "ab"))
        except:
          print("damp error:" + str(i))
      except :
        print("ERROR: "+ str(i))


# 「画像とラベル」のデータセットを保存
pickle.dump(result, open(save_file, "ab"))
plt.show()

### learning

In [2]:
import keras

import numpy as np
import cv2, pickle, dill
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

from tensorflow.keras import utils as np_utils


In [7]:
# データファイルと画像サイズの指定

data_file = OUTPUT_DIR  + "/pickle/ETL9B.pickle" 
im_size = 64
out_size = 3036 # 文字の数
im_color = 1
in_shape = (im_size, im_size, im_color)

In [13]:
# 保存した画像データ一覧を読み込む
data = pickle.load(open(data_file, "rb"))


In [8]:
#dillファイルをお見込む場合
dill_file = OUTPUT_DIR + "/pickle/ETL9B.dill" 
data = dill.load(open(dill_file, "rb"))
print(len(data))

In [ ]:
###test
for d in data:
  (num, img) = d
  print(num)
  if num == 0: continue
  img = cv2.imread(img)
  plt.imshow(img)
  plt.show()

In [ ]:
# 画像データを0-1の範囲に直す
y = []
x = []
for d in data:
  (num, img) = d
  print(num)
  img = img.astype('float').reshape(im_size, im_size, im_color) / 255
  y.append(np_utils.to_categorical(num, out_size))
  x.append(img)
x = np.array(x)
y = np.array(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, shuffle=True)

In [3]:
#モデル構築
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=in_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(out_size))
model.add(Activation('softmax'))



NameError: ignored

In [ ]:
#コンパイル
model.compile(
  loss='categorical_crossentropy',
  optimizer= 'adam',
  metrics=['accuracy'])

hist = model.fit(
  x_train, y_train,
  batch_size=2048, epochs=20000,verbose=1,
    validation_data=(x_test, y_test))

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print("正解率 ", score[1], "loss ", score[0])

plt.plot(hist.history['acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()

model.save('ETL9-model.h5')
model.save_weights('ETL9-weights.h5')